In [1]:
import importlib

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
model_module = importlib.import_module("handyrl.model")
env_module = importlib.import_module("handyrl.envs.kaggle.hungry_geese")

Loading environment football failed: No module named 'gfootball'


In [3]:
# pretrained_model_path = "ds/models/first_stage_2462.pth"
# weights = torch.load(pretrained_model_path)

In [4]:
e = env_module.Environment()
e.reset()

In [5]:
obs = e.observation()
obs = torch.from_numpy(obs.reshape(1, 80)).clone()
obs_ = torch.cat([obs, obs], dim=0)
print(f"size: {obs_.size()}, type: {obs_.dtype}")

size: torch.Size([2, 80]), type: torch.int64


In [6]:
e.obs_list[-1]

[{'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60,
   'step': 0,
   'geese': [[23], [76], [18], [44]],
   'food': [63, 39],
   'index': 0},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 1},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 2},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 3},
  'status': 'ACTIVE'}]

In [7]:
input_ = torch.randn(2, 80)
print(f"size: {input_.size()}, type: {input_.dtype}")

size: torch.Size([2, 80]), type: torch.float32


In [8]:
obs

tensor([[ 1,  1,  1,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  4,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         18,  1,  1,  1,  1,  5,  1,  1,  1,  1,  1,  1,  1, 18,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  0,  0,  0]])

In [9]:
net = env_module.GeeseNetIMO(e)

In [10]:
# net.geese_net.load_state_dict(weights, strict=False)

In [11]:
# for param in net.geese_net.parameters():
#     param.requires_grad = False

In [12]:
# pytorch_total_params
params = sum(p.numel() for p in net.parameters())
print(f"{params:,}")

695,881


In [13]:
# pytorch_total_params (trainable)
params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f"{params:,}")

695,881


In [14]:
net

GeeseNetIMO(
  (encoder): GeeseEncoder(
    (embed): Embedding(20, 2)
  )
  (blocks): ModuleList(
    (0): GeeseBlock(
      (attention): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=160, out_features=160, bias=True)
      )
    )
    (1): GeeseBlock(
      (attention): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=160, out_features=160, bias=True)
      )
    )
    (2): GeeseBlock(
      (attention): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=160, out_features=160, bias=True)
      )
    )
    (3): GeeseBlock(
      (attention): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=160, out_features=160, bias=True)
      )
    )
    (4): GeeseBlock(
      (attention): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=160, out_features=160, bias=True)
      )
    )
    (5): GeeseBlock(
      (attention): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=160, out_featu

In [15]:
out = net(obs_)
print(f"size: {out['policy'].size()}, type: {out['policy'].dtype}")

size: torch.Size([2, 4]), type: torch.float32


In [16]:
out

{'policy': tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.]], grad_fn=<MmBackward>),
 'value': tensor([[-0.0193],
         [-0.0193]], grad_fn=<TanhBackward>)}